In [6]:
from math_go_brrr import brrr
from time import time
from types import FunctionType

## Comparing the Python and LLVM JIT version

In [7]:
@brrr
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b



start = time()
llvm_out = foo(10_000_000)
print(f"LLVM JIT Compiled |", time() - start)

# foo.original_func is our plain-old Python function
assert isinstance(foo.original_func, FunctionType)

start = time()
python_out = foo.original_func(10_000_000)
print(f"Regular Python    |", time() - start)

LLVM JIT Compiled | 0.02686166763305664
Regular Python    | 0.37555527687072754


## Testing various optimization levels

In [8]:
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b


for lvl in ["none", "less", "default", "aggressive"]:
    foo_compiled = brrr(optimization=lvl)(foo)

    start = time()
    llvm_out = foo_compiled(10_000_000)
    print(f"LLVM opt = {lvl:<10} |", time() - start)

start = time()
python_out = foo(10_000_000)
print(f"Regular Python        |", time() - start)

LLVM opt = none       | 0.024113893508911133
LLVM opt = less       | 0.02508378028869629
LLVM opt = default    | 0.0253751277923584
LLVM opt = aggressive | 0.02697300910949707
Regular Python        | 0.37921977043151855


## Testing the performance with different input sizes

Since the calling of the JIT function jumps through a few more hoops internally and has to convert python types to LLVM types, the JIT version is slower at small input sizes. However, it scales much better than python with larger input sizes.

In [9]:
@brrr
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b

# the first run does very badly,
# and I assume because of the JIT it might
# only compile when it's called the first time
foo(1)

for count in [1, 10, 100, 10_000, 1_000_000]:
  print(f"Count: {count}")
  start = time()
  llvm_out = foo(count)
  print(f"  LLVM JIT Compiled | {time() - start:.7f}")

  # foo.original_func is our plain-old Python function
  assert isinstance(foo.original_func, FunctionType)

  start = time()
  python_out = foo.original_func(count)
  print(f"  Regular Python    | {time() - start:.7f}")

Count: 1
  LLVM JIT Compiled | 0.0000319
  Regular Python    | 0.0000031
Count: 10
  LLVM JIT Compiled | 0.0000088
  Regular Python    | 0.0000021
Count: 100
  LLVM JIT Compiled | 0.0000179
  Regular Python    | 0.0000048
Count: 10000
  LLVM JIT Compiled | 0.0000279
  Regular Python    | 0.0003471
Count: 1000000
  LLVM JIT Compiled | 0.0023050
  Regular Python    | 0.0375988


In [10]:
@brrr
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b

from math_go_brrr import foo as foo_rust


start = time()
llvm_out = foo(10_000_000)
print(f"LLVM JIT Compiled |", time() - start)

# foo.original_func is our plain-old Python function
assert isinstance(foo.original_func, FunctionType)

start = time()
python_out = foo.original_func(10_000_000)
print(f"Regular Python    |", time() - start)

start = time()
rust_out = foo_rust(10_000_000)
print(f"Rust Compiled     |", time() - start)


LLVM JIT Compiled | 0.024518966674804688
Regular Python    | 0.37351512908935547
Rust Compiled     | 0.012889862060546875


So it looks like, while the JIT version is 10x faster than python, writing it in Rust instead would save another 2x.